# Notes on the XFitLLSGUI (v0.3)

## Quick background

This GUI is based on one originally built by JXP and JMO to
enable the manual fitting of LLS in absorption line spectra.
That GUI was applied to real and mock data for a number of 
LLS surveys.

This Python-based incantation follows that prescription, by
and large.

## Launching the GUI

The GUI is instantiated through the XFitLLSGUI class
which combines several widgets.  I give a few examples
below.

### From within ipython 
Need to pass the call a spectrum file or the XSpectrum1D 
object itself.

**WARNING:** -- You cannot use inline matplotlib (I think)

In [ ]:
from xastropy.xguis import spec_guis as xxsg
reload(xxsg)
#
spec_fil = '/Users/xavier/VLT/XShooter/LP/idl_reduced_frames/0952-0115_uvb_coadd_vbin_flx.fits'
xxsg.run_fitlls(spec_fil)

### Command line call -- Recommended

Run spec_guis with the '4' flag

I recommend you generate an alias like this:

    alias xfitlls 'python ~/xastropy/xastropy/xguis/spec_guis.py 4'

Then it is as simple as:

    unix> xfitlls 0952-0115_uvb_coadd_vbin_flx.fits
    
Here is the current help:

    usage: spec_guis.py [-h] [-out_file OUT_FILE] [-smooth SMOOTH] flag in_file

    Parser for XFitLLSGUI

    positional arguments:
      flag                GUI flag (ignored)
      in_file             Spectral file

    optional arguments:
      -h, --help          show this help message and exit
      -out_file OUT_FILE  LLS Fit file
      -smooth SMOOTH      Smoothing (pixels)
      -lls_fit_file LLS_FIT_FILE
                          Input LLS Fit file

---
## Fiddling about with the GUI

### Basics
The GUI has been built to enable a user to overlay a series
of LLS 'models' on a spectrum, including a continuum.  The
LLS is modeled by the Lyman series to Lyman-30, with a traditional
N,b,z parameterization and then continuum opacity beyond the Limit.
There is a "kludge" implemented to cover the gap between Lyman-30
and the LL.

Here is an overview of using the GUI:

(1) Most of the action occurs in the plot window.  

(2) The obvious exceptions are when you wish to change
between LLS for fiddling their parameters.  For that,
you need to select the LLS in the list widget.

(3) You can modify redshift, N_HI or bval in the boxes provided.

(4) Lastly, you can Write, Write+Quit, or Quit with the 
boxes at the bottom right.

**Note:** You have to click in the plot window for key strokes
to take effect.  This includes after actions that engage the LLS list widget.

### Navigating
These key strokes for navigating the spectral window
are identical to those for the XSpecGUI.
Please refer to its documentation.  

### LLS commands

* A -- Add a new LLS
* D -- Delete the selected LLS
* n/N -- Decrease/increase the N_HI column by 0.05dex of the selected LLS
* v/V -- Decrease/increase the b-value by 2 km/s
* a -- Set the LLS redshift such that the cursor is at Lya
* L -- Set the LLS redshift such that the cursor is at the LL
* @ -- Toggle marking strong lines (e.g. CII 1334, CIV)

### Continuum commands

* C -- Fiddle with the normalization
* 1/2 -- Tilt the continuum with a power-law pivoting about the median wavelength value


### Other

* 6,7,8,9 -- Add a Lya forest line (and Lyb,Lyg) at this location with NHI=12,13,14, or 15

---
## Output/Input

When you Write or Write+Quit, a JSON output file is generated
that saves the LLS parameters, the continuum parameters and a
few other bits and pieces.  Here is an example:

{
    "LLS": {
        "1": {
            "NHI": 17.3,
            "bval": 42.0,
            "z": 4.1348662134175811
        }
    },
    "conti": {
        "Norm": 6.6073753071076802e-18,
        "piv_wv": 4166.4861195151479,
        "tilt": 0.0
    },
    "smooth": 2.5,
    "spec_file": "0952-0115_uvb_coadd_vbin_flx.fits"
}


### Reading a previous fit

Just pass the JSON file to lls_file, e.g.

    unix> xfitlls 0952-0115_uvb_coadd_vbin_flx.fits -lls_fit_file=tmp.json

## Testing

In [3]:
#LL Opacity
from xastropy.atomic import ionization as xatomi
wv_rest = 911.5*u.AA
energy = wv_rest.to(u.eV, equivalencies=u.spectral())
tau_LL = (10.**17.5 / u.cm**2) * xatomi.photo_cross(1,1,energy)
print(tau_LL)
# XIDL gives 2.0060005 -- Close enough
#tau_eng = c.h*c.c/(tau_wv*1e-8) / c.ev ; eV
#tau_LL = 10.^(16+tau_id*delta_N) * x_photocross(1,1, tau_eng) 

2.0059970185


In [1]:
#Lyman-a
#line = x_setline(1215.6701)
#line.N=17.5
#line.b=20.
# wave = 1200. + findgen(40000)*0.001
# vgt = x_voigt(wave,line,tau=tau,/nosmooth)
# tau peaks at 11978.7
from linetools.spectralline import AbsLine
lya = AbsLine('HI 1215')
lya.attrib['N'] = 17.5
lya.attrib['b'] = 20.*u.km/u.s

linetools.lists.parse: Reading linelist --- 
   /Users/xavier/local/Python/linetools/linetools/data/lines/morton03_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/xavier/local/Python/linetools/linetools/data/lines/morton00_table2.fits.gz
linetools.lists.parse: Reading linelist --- 
   /Users/xavier/local/Python/linetools/linetools/data/lines/verner94_tab6.fits
linetools.lists.parse: Reading linelist --- 
   /Users/xavier/local/Python/linetools/linetools/data/lines/EUV_lines.ascii
read_sets: Using set file -- 
  /Users/xavier/local/Python/linetools/linetools/lists/sets/llist_v0.3.ascii


In [2]:
lya.data

{'A': <Quantity 626500000.0 1 / s>,
 'Am': 0,
 'Ej': <Quantity 0.0 1 / cm>,
 'Ek': <Quantity 2259.163 1 / cm>,
 'Ex': <Quantity 0.0 1 / cm>,
 'Jj': 0.0,
 'Jk': 0.0,
 'Ref': 'Morton2003',
 'Z': 1,
 'col0': masked,
 'col6': masked,
 'el': 0,
 'f': 0.41639999999999999,
 'gamma': <Quantity 626500000.0 1 / s>,
 'gj': 2,
 'gk': 6,
 'group': 1,
 'ion': 1,
 'mol': '',
 'name': 'HI 1215',
 'nj': 0,
 'nk': 0,
 'wrest': <Quantity 1215.67 Angstrom>}

In [10]:
from xastropy.spec import voigt as xsv
from linetools.spectra.xspectrum1d import XSpectrum1D
reload(xsv)
wv = np.arange(1200.,1240,0.001)
npix = len(wv)
spec = XSpectrum1D.from_tuple((wv, np.ones(npix)))
tau = xsv.voigt_model(spec,lya,flg_ret=2)
# Max at ~6760!
# Now at 11974. Thank goodness..

In [11]:
xdb.xplot(wv,tau)

In [7]:
# Voigt
# IDL for print,voigt(0.0003030379,0.)=0.999658
avoigt = 0.0003030378771933264 # Same a in each evaluation
xsv.voigtking(np.array([0.,1.]),avoigt)

array([ 0.99965815,  0.36790545])

In [9]:
np.sqrt(np.pi)

1.7724538509055159